In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
data = pd.read_csv("/kaggle/input/chicken-disease-1/train_data.csv")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8067 entries, 0 to 8066
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   images  8067 non-null   object
 1   label   8067 non-null   object
dtypes: object(2)
memory usage: 126.2+ KB


In [5]:
data.head()

,images,label
0,salmo.1558.jpg,Salmonella
1,cocci.1866.jpg,Coccidiosis
2,cocci.171.jpg,Coccidiosis
3,salmo.1484.jpg,Salmonella
4,ncd.100.jpg,New Castle Disease


In [6]:
data['label'].value_counts()

label
Salmonella            2625
Coccidiosis           2476
Healthy               2404
New Castle Disease     562
Name: count, dtype: int64

In [7]:
import os
import cv2

directory = "/kaggle/input/chicken-disease-1/Train"

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith('.jpg'):
            img = cv2.imread(os.path.join(folder, filename))
            if img is not None:
                images.append(img)
            else:
                continue
    return images


images = load_images_from_folder(directory)

In [8]:
images = np.array(images)

In [9]:
images[0].shape

(224, 224, 3)

In [10]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
data[['label']] = encoder.fit_transform(data[['label']])
#data['label'] = data['label'].astype(int)

In [11]:
data['label']

0       3.0
1       0.0
2       0.0
3       3.0
4       2.0
       ... 
8062    1.0
8063    3.0
8064    2.0
8065    3.0
8066    1.0
Name: label, Length: 8067, dtype: float64

In [12]:
labels = data['label'].tolist()
labels = np.array(labels)
labels = labels.flatten()

In [13]:
from sklearn.utils import shuffle
images , labels = shuffle(images , labels)

In [14]:
from sklearn.model_selection import train_test_split
X_train , X_valid , y_train , y_valid = train_test_split(images , labels ,test_size=0.2,random_state=42)

In [15]:
leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2)

In [16]:
tf.random.set_seed(42)
tf.keras.backend.clear_session()

from functools import partial

DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, padding="same",
                        activation=leaky_relu, kernel_initializer="he_normal")
model = tf.keras.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[224, 224, 3]),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation=leaky_relu,
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=64, activation=leaky_relu,
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=4, activation="softmax")
])

In [17]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=15,
                    validation_data=(X_valid, y_valid))

Epoch 1/15
202/202 [==============================] - 72s 265ms/step - loss: 324.8330 - accuracy: 0.2867 - val_loss: 5.0219 - val_accuracy: 0.3147
Epoch 2/15
202/202 [==============================] - 50s 250ms/step - loss: 9.1975 - accuracy: 0.2934 - val_loss: 2.1379 - val_accuracy: 0.3172
Epoch 3/15
202/202 [==============================] - 52s 256ms/step - loss: 6.1266 - accuracy: 0.3073 - val_loss: 2.7184 - val_accuracy: 0.3154
Epoch 4/15
202/202 [==============================] - 52s 256ms/step - loss: 4.2498 - accuracy: 0.3231 - val_loss: 2.0197 - val_accuracy: 0.3055
Epoch 5/15
202/202 [==============================] - 52s 256ms/step - loss: 3.2455 - accuracy: 0.3327 - val_loss: 1.5725 - val_accuracy: 0.3129
Epoch 6/15
202/202 [==============================] - 52s 257ms/step - loss: 2.5449 - accuracy: 0.3598 - val_loss: 1.3892 - val_accuracy: 0.3141
Epoch 7/15
202/202 [==============================] - 52s 258ms/step - loss: 2.1113 - accuracy: 0.3811 - val_loss: 1.6148 - val_

In [18]:
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="same", kernel_initializer="he_normal",
                        use_bias=False)

class ResidualUnit(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation=leaky_relu, **kwargs):
        super().__init__(**kwargs)
        self.activation = tf.keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            tf.keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            tf.keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                tf.keras.layers.BatchNormalization()
            ]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [19]:
tf.random.set_seed(42)
tf.keras.backend.clear_session()

model = tf.keras.Sequential([
    DefaultConv2D(64, kernel_size=7, strides=2, input_shape=[224,224, 3]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation(leaky_relu),
    tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"),
])

residual_blocks = [3, 4, 6, 3]  # ResNet-50 configuration

prev_filters = 64
for filters, blocks in zip([64, 128, 256, 512], residual_blocks):
    for _ in range(blocks):
        if filters != prev_filters:
            strides = 2
        else:
            strides = 1
        model.add(ResidualUnit(filters, strides=strides))
        prev_filters = filters

model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(4, activation="softmax"))

In [20]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=15,
                    validation_data=(X_valid, y_valid))

Epoch 1/15
202/202 [==============================] - 76s 215ms/step - loss: 1.4495 - accuracy: 0.3161 - val_loss: 2.8378 - val_accuracy: 0.3191
Epoch 2/15
202/202 [==============================] - 40s 197ms/step - loss: 1.3185 - accuracy: 0.3119 - val_loss: 3.6317 - val_accuracy: 0.3222
Epoch 3/15
202/202 [==============================] - 40s 198ms/step - loss: 1.3155 - accuracy: 0.3003 - val_loss: 1.3289 - val_accuracy: 0.3123
Epoch 4/15
202/202 [==============================] - 40s 197ms/step - loss: 1.3153 - accuracy: 0.3143 - val_loss: 1.2830 - val_accuracy: 0.3203
Epoch 5/15
202/202 [==============================] - 40s 198ms/step - loss: 1.3094 - accuracy: 0.3161 - val_loss: 1.3286 - val_accuracy: 0.3141
Epoch 6/15
202/202 [==============================] - 40s 197ms/step - loss: 1.3056 - accuracy: 0.3178 - val_loss: 1.3051 - val_accuracy: 0.3011
Epoch 7/15
202/202 [==============================] - 40s 197ms/step - loss: 1.3038 - accuracy: 0.3119 - val_loss: 1.6202 - val_ac

In [21]:
tf.random.set_seed(42)
tf.keras.backend.clear_session()

base_model = tf.keras.applications.xception.Xception(weights="imagenet",
                                                     include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(4, activation="softmax")(avg)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

83683744/83683744 [==============================] - 1s 0us/step


In [22]:
for layer in base_model.layers:
    layer.trainable = False

In [23]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
202/202 [==============================] - 43s 190ms/step - loss: 16836.2676 - accuracy: 0.2969 - val_loss: 12701.3232 - val_accuracy: 0.2993
Epoch 2/5
202/202 [==============================] - 34s 170ms/step - loss: 21265.8672 - accuracy: 0.2930 - val_loss: 15817.6484 - val_accuracy: 0.3055
Epoch 3/5
202/202 [==============================] - 34s 169ms/step - loss: 21664.6562 - accuracy: 0.2954 - val_loss: 29984.6113 - val_accuracy: 0.3055
Epoch 4/5
202/202 [==============================] - 34s 171ms/step - loss: 17434.5605 - accuracy: 0.3051 - val_loss: 36243.2500 - val_accuracy: 0.3123
Epoch 5/5
202/202 [==============================] - 34s 170ms/step - loss: 23789.9043 - accuracy: 0.3160 - val_loss: 53137.6211 - val_accuracy: 0.3141


In [24]:
for indices in zip(range(33), range(33, 66), range(66, 99), range(99, 132)):
    for idx in indices:
        print(f"{idx:3}: {base_model.layers[idx].name:22}", end="")
    print()

  0: input_1                33: block4_pool            66: block8_sepconv1_act    99: block11_sepconv2_act  
  1: block1_conv1           34: batch_normalization_2  67: block8_sepconv1       100: block11_sepconv2      
  2: block1_conv1_bn        35: add_2                  68: block8_sepconv1_bn    101: block11_sepconv2_bn   
  3: block1_conv1_act       36: block5_sepconv1_act    69: block8_sepconv2_act   102: block11_sepconv3_act  
  4: block1_conv2           37: block5_sepconv1        70: block8_sepconv2       103: block11_sepconv3      
  5: block1_conv2_bn        38: block5_sepconv1_bn     71: block8_sepconv2_bn    104: block11_sepconv3_bn   
  6: block1_conv2_act       39: block5_sepconv2_act    72: block8_sepconv3_act   105: add_9                 
  7: block2_sepconv1        40: block5_sepconv2        73: block8_sepconv3       106: block12_sepconv1_act  
  8: block2_sepconv1_bn     41: block5_sepconv2_bn     74: block8_sepconv3_bn    107: block12_sepconv1      
  9: block2_sepconv

In [25]:
for layer in base_model.layers[56:]:
    layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=15,
                    validation_data=(X_valid, y_valid))

Epoch 1/15
202/202 [==============================] - 78s 337ms/step - loss: 4.5336 - accuracy: 0.3240 - val_loss: 1.3752 - val_accuracy: 0.3222
Epoch 2/15
202/202 [==============================] - 65s 321ms/step - loss: 1.2790 - accuracy: 0.3219 - val_loss: 1.2605 - val_accuracy: 0.3203
Epoch 3/15
202/202 [==============================] - 64s 316ms/step - loss: 1.2788 - accuracy: 0.3251 - val_loss: 1.2611 - val_accuracy: 0.3061
Epoch 4/15
202/202 [==============================] - 63s 313ms/step - loss: 1.2790 - accuracy: 0.3253 - val_loss: 1.2597 - val_accuracy: 0.3209
Epoch 5/15
202/202 [==============================] - 63s 314ms/step - loss: 1.2787 - accuracy: 0.3268 - val_loss: 1.2593 - val_accuracy: 0.3067
Epoch 6/15
202/202 [==============================] - 63s 311ms/step - loss: 1.2784 - accuracy: 0.3216 - val_loss: 1.2595 - val_accuracy: 0.3209
Epoch 7/15
202/202 [==============================] - 63s 310ms/step - loss: 1.2788 - accuracy: 0.3234 - val_loss: 1.2603 - val_ac